<a href="https://colab.research.google.com/github/mickeykubo/analytics_book/blob/master/integer_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python言語によるビジネスアナリティクス

## 実務家のための最適化，統計分析，機械学習（近代科学社）

## PuLPとGurobi/Pythonによる混合整数最適化問題

* 簡単な例題
* 多制約ナップサック問題


In [1]:
!pip install mypulp

## 例題 パズル（整数最適化）

鶴と亀と蛸が何匹かずついる
頭の数を足すと $32$，足の数を足すと $80$ になる．
亀と蛸の数の和を一番小さくするような匹数を求めよ．

\begin{array}{l c c }
  \mbox{minimize}    &  y + z          \\
  \mbox{subject to}  & x + y + z  = 32 \\
                     & 2x +4y +8z = 80 \\
               & x,y,z  \mbox{ は非負の整数} 
\end{array}



In [2]:
#from gurobipy import *
from mypulp import *
model = Model("puzzle")
x = model.addVar(vtype="I", name="x")
y = model.addVar(vtype="I", name="y")
z = model.addVar(vtype="I", name="z")
model.update()

model.addConstr(x + y + z == 32, "Heads")
model.addConstr(2*x + 4*y + 8*z == 80, "Legs")

model.setObjective(y + z, GRB.MINIMIZE)

model.optimize()

print("Opt. Val.=",model.ObjVal)
print("(x,y,z)=",(x.X,y.X,z.X))

Opt. Val.= 4.0
(x,y,z)= (28.0, 2.0, 2.0)


## 例題 多制約ナップサック問題（0-1整数最適化）

* $n$ 個のアイテム，$m$本の制約
* 各々のアイテム $j=1,2,\ldots,n$ の価値 $v_j ~(\geq 0)$，
* アイテム $j$ の制約 $i=1,2,\ldots,m$ に対する重み $a_{ij}~(\geq 0)$
* 制約 $i$ に対する制約の上限値 $b_i~(\geq 0)$ 



* アイテムの番号の集合 $I =\{1,2,\ldots,n \}$
* 制約の番号の集合 $J =\{1,2,\ldots,m\}$ 
* アイテム $j$ をナップサックに詰めるとき $1$，
それ以外のとき $0$ になる $0$-$1$ 変数 $x_j$

定式化

\begin{array}{l l l}
   \mbox{maximize}     & \displaystyle\sum_{j \in J} v_j x_j          &  \\
   \mbox{subject to}  &  \displaystyle\sum_{j \in J} a_{ij} x_j \leq b_i & \forall i \in I    \\
                       & x_j \in \{0,1\} &     \forall j \in J
\end{array}

In [3]:
#from gurobipy import *
from mypulp import *

J,v = multidict({1:16, 2:19, 3:23, 4:28})
a = {(1,1):2,    (1,2):3,    (1,3):4,    (1,4):5,
     (2,1):3000, (2,2):3500, (2,3):5100, (2,4):7200,
    }
I,b = multidict({1:7, 2:10000})
    
model = Model('mkp')
x = {}
for j in J:
    x[j] = model.addVar(vtype='B', name='x({0})'.format(j))
model.update()
     
for i in I:
    model.addConstr(quicksum(a[i,j]*x[j] for j in J) <= b[i], 'Capacity({0})'.format(i))

model.setObjective(quicksum(v[j]*x[j] for j in J), GRB.MAXIMIZE)

model.optimize()
print('Optimal value=',model.ObjVal)

EPS = 1.e-6
for v in model.getVars():
    if v.X > EPS:
        print( v.VarName,v.X)

Optimal value= 42.0
x(2) 1.0
x(3) 1.0
